# 05. Trực quan hóa Lộ trình (Tour Visualization) - Đầy đủ

**Mục tiêu:** Notebook này dùng để tạo và lưu lại hình ảnh trực quan của các lộ trình (tours) được tạo bởi các thuật toán khác nhau.

Phiên bản này đã được cập nhật để chạy và trực quan hóa **tất cả 10 thuật toán** của dự án.

Chúng ta sẽ:
1.  Chọn một vấn đề (ví dụ: `berlin52`).
2.  Chạy tất cả 10 thuật toán (với điều kiện cho các thuật toán Exact).
3.  Sử dụng `utils.visualizer.plot_tour` với tham số `save_path` để lưu từng hình ảnh vào thư mục `/results/tours/`.

## 1. Thiết lập & Imports

Import các thư viện và các module thuật toán/tiện ích cần thiết.

In [ ]:
import sys
import os
import time
import numpy as np
import matplotlib.pyplot as plt

# --- Cấu hình sys.path --- 
project_root = os.path.normpath(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Đã thêm '{project_root}' vào sys.path")

# --- Import Utils --- 
from utils import data_loader
from utils import evaluator
from utils import visualizer

# --- Import 10 Thuật toán --- 
# Exact
from algorithms import brute_force
from algorithms import held_karp
# Heuristics Cổ điển & Xấp xỉ
from algorithms import nearest_neighbor
from algorithms import nearest_insertion
from algorithms import christofides
# Cải tiến
from algorithms import two_opt
# Metaheuristics
from algorithms import simulated_annealing
from algorithms import genetic_algorithm
from algorithms import ant_colony
from algorithms import tabu_search

# Cấu hình chung
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)
print("Các module đã được import.")

## 2. Cấu hình & Tải Dữ liệu

Chọn vấn đề và các thư mục. Chúng ta cũng sẽ định nghĩa các tham số cho các thuật toán Metaheuristic.

In [ ]:
# --- VẤN ĐỀ ĐỂ TRỰC QUAN HÓA ---
PROBLEM_NAME = 'berlin52' # Thử đổi thành 'test_10' để chạy Brute Force

# --- ĐƯỜNG DẪN --- 
DATA_DIR = '../data'
# Chúng ta sẽ tạo một thư mục con mới để lưu các hình ảnh này
OUTPUT_DIR = '../results/tours'

# Tạo thư mục lưu hình ảnh nếu chưa có
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Đã tạo thư mục: {OUTPUT_DIR}")

# --- Tải Dữ liệu Vấn đề ---
try:
    problem_data = data_loader.load_problem(PROBLEM_NAME, DATA_DIR)
    matrix = problem_data['matrix']
    coords = problem_data['coords']
    opt_tour = problem_data['optimum_tour']
    dim = problem_data['dimension']
    opt_cost = evaluator.calculate_tour_cost(opt_tour, matrix)
    
    print(f"Đã tải thành công {PROBLEM_NAME} (N={problem_data['dimension']})")
    print(f"Chi phí tối ưu: {opt_cost}")
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file cho {PROBLEM_NAME}.")
    # Dừng notebook nếu không tải được
    raise

# --- Tham số cho Metaheuristics (lấy từ 03_experiments) ---
SA_PARAMS = {
    'initial_temp': 10000.0,
    'cooling_rate': 0.999,
    'min_temp': 0.1,
    'max_iterations_per_temp': 100
}

GA_PARAMS = {
    'population_size': 100,
    'num_generations': 500,
    'mutation_rate': 0.02,
    'elite_size': 10,
    'tournament_k': 5
}

ACO_PARAMS = {
    'num_ants': dim, # Số kiến bằng số thành phố
    'num_iterations': 100,
    'alpha': 1.0, 
    'beta': 5.0,
    'evaporation_rate': 0.5,
    'pheromone_deposit': 100.0
}

TS_PARAMS = {
    'max_iterations': 2000,
    'tabu_tenure': int(dim * 0.5) # Nhiệm kỳ cấm ~ N/2
}

## 3. Vẽ & Lưu Lộ trình Tối ưu (Optimum)

Đầu tiên, hãy vẽ giải pháp "hoàn hảo" để làm cơ sở so sánh.

In [ ]:
if opt_tour:
    print(f"Đang vẽ lộ trình Tối ưu (Cost: {opt_cost})...")
    
    save_path = os.path.join(OUTPUT_DIR, f"{PROBLEM_NAME}_0_optimum.png")
    
    visualizer.plot_tour(
        tour=opt_tour,
        coords=coords,
        title=f"Optimum Tour ({PROBLEM_NAME})\nCost: {opt_cost:.0f}",
        save_path=save_path
    )
else:
    print(f"Không tìm thấy tour tối ưu cho {PROBLEM_NAME}.")

## 4. Chạy các Thuật toán & Lưu kết quả

Bây giờ, chúng ta sẽ lần lượt chạy từng thuật toán, đo thời gian và lưu lại hình ảnh lộ trình.

### 4.1. Brute Force (N <= 10)

In [ ]:
if dim <= 10:
    print("Đang chạy Brute Force...")
    start_time = time.time()
    bf_tour, bf_cost = brute_force.solve(matrix)
    exec_time = time.time() - start_time
    print(f"Brute Force hoàn thành trong {exec_time:.3f}s. Cost: {bf_cost:.0f}")

    save_path = os.path.join(OUTPUT_DIR, f"{PROBLEM_NAME}_1_brute_force.png")
    visualizer.plot_tour(bf_tour, coords, f"Brute Force Tour ({PROBLEM_NAME})\nCost: {bf_cost:.0f}", save_path)
else:
    print(f"Bỏ qua Brute Force vì N={dim} > 10.")

### 4.2. Held-Karp (N <= 21)

In [ ]:
if dim <= 21:
    print("Đang chạy Held-Karp...")
    start_time = time.time()
    hk_tour, hk_cost = held_karp.solve(matrix)
    exec_time = time.time() - start_time
    print(f"Held-Karp hoàn thành trong {exec_time:.3f}s. Cost: {hk_cost:.0f}")

    save_path = os.path.join(OUTPUT_DIR, f"{PROBLEM_NAME}_2_held_karp.png")
    visualizer.plot_tour(hk_tour, coords, f"Held-Karp Tour ({PROBLEM_NAME})\nCost: {hk_cost:.0f}", save_path)
else:
    print(f"Bỏ qua Held-Karp vì N={dim} > 21.")

### 4.3. Nearest Neighbor (NN)

Tiếp theo, một giải pháp Heuristic "ngây thơ".

In [ ]:
print("Đang chạy Nearest Neighbor...")
start_time = time.time()
nn_tour, nn_cost = nearest_neighbor.solve(matrix, start_node=0)
exec_time = time.time() - start_time
print(f"NN hoàn thành trong {exec_time:.3f}s. Cost: {nn_cost:.0f}")

# Lưu hình ảnh
save_path = os.path.join(OUTPUT_DIR, f"{PROBLEM_NAME}_3_nearest_neighbor.png")

visualizer.plot_tour(
    tour=nn_tour,
    coords=coords,
    title=f"Nearest Neighbor Tour ({PROBLEM_NAME})\nCost: {nn_cost:.0f}",
    save_path=save_path
)

### 4.4. Nearest Insertion (NI)

In [ ]:
print("Đang chạy Nearest Insertion...")
start_time = time.time()
ni_tour, ni_cost = nearest_insertion.solve(matrix, start_node=0)
exec_time = time.time() - start_time
print(f"NI hoàn thành trong {exec_time:.3f}s. Cost: {ni_cost:.0f}")

save_path = os.path.join(OUTPUT_DIR, f"{PROBLEM_NAME}_4_nearest_insertion.png")
visualizer.plot_tour(ni_tour, coords, f"Nearest Insertion Tour ({PROBLEM_NAME})\nCost: {ni_cost:.0f}", save_path)

### 4.5. Christofides

In [ ]:
print("Đang chạy Christofides...")
start_time = time.time()
chris_tour, chris_cost = christofides.solve(matrix)
exec_time = time.time() - start_time
print(f"Christofides hoàn thành trong {exec_time:.3f}s. Cost: {chris_cost:.0f}")

save_path = os.path.join(OUTPUT_DIR, f"{PROBLEM_NAME}_5_christofides.png")
visualizer.plot_tour(chris_tour, coords, f"Christofides Tour ({PROBLEM_NAME})\nCost: {chris_cost:.0f}", save_path)

### 4.6. 2-Opt (cải thiện từ NN)

Cải thiện giải pháp NN bằng 2-Opt.

In [ ]:
print("Đang chạy 2-Opt (bắt đầu từ NN)...")
start_time = time.time()
two_opt_tour, two_opt_cost = two_opt.solve(matrix, initial_tour=nn_tour)
exec_time = time.time() - start_time
print(f"2-Opt hoàn thành trong {exec_time:.3f}s. Cost: {two_opt_cost:.0f}")

# Lưu hình ảnh
save_path = os.path.join(OUTPUT_DIR, f"{PROBLEM_NAME}_6_two_opt.png")

visualizer.plot_tour(
    tour=two_opt_tour,
    coords=coords,
    title=f"2-Opt (from NN) Tour ({PROBLEM_NAME})\nCost: {two_opt_cost:.0f}",
    save_path=save_path
)

### 4.7. Simulated Annealing (SA)

In [ ]:
print("Đang chạy Simulated Annealing (SA)...")
start_time = time.time()
sa_tour, sa_cost = simulated_annealing.solve(matrix, initial_tour=nn_tour, **SA_PARAMS)
exec_time = time.time() - start_time
print(f"SA hoàn thành trong {exec_time:.3f}s. Cost: {sa_cost:.0f}")

save_path = os.path.join(OUTPUT_DIR, f"{PROBLEM_NAME}_7_simulated_annealing.png")
visualizer.plot_tour(sa_tour, coords, f"Simulated Annealing Tour ({PROBLEM_NAME})\nCost: {sa_cost:.0f}", save_path)

### 4.8. Genetic Algorithm (GA)

Cuối cùng, chạy một Metaheuristic mạnh mẽ (có thể mất vài giây).

In [ ]:
print("Đang chạy Genetic Algorithm (GA)...")
start_time = time.time()
ga_tour, ga_cost = genetic_algorithm.solve(matrix, **GA_PARAMS)
exec_time = time.time() - start_time
print(f"GA hoàn thành trong {exec_time:.3f}s. Cost: {ga_cost:.0f}")

# Lưu hình ảnh
save_path = os.path.join(OUTPUT_DIR, f"{PROBLEM_NAME}_8_genetic_algorithm.png")

visualizer.plot_tour(
    tour=ga_tour,
    coords=coords,
    title=f"Genetic Algorithm Tour ({PROBLEM_NAME})\nCost: {ga_cost:.0f}",
    save_path=save_path
)

### 4.9. Ant Colony Optimization (ACO)

In [ ]:
print("Đang chạy Ant Colony Optimization (ACO)...")
start_time = time.time()
aco_tour, aco_cost = ant_colony.solve(matrix, **ACO_PARAMS)
exec_time = time.time() - start_time
print(f"ACO hoàn thành trong {exec_time:.3f}s. Cost: {aco_cost:.0f}")

save_path = os.path.join(OUTPUT_DIR, f"{PROBLEM_NAME}_9_ant_colony.png")
visualizer.plot_tour(aco_tour, coords, f"Ant Colony Tour ({PROBLEM_NAME})\nCost: {aco_cost:.0f}", save_path)

### 4.10. Tabu Search (TS)

In [ ]:
print("Đang chạy Tabu Search (TS)...")
start_time = time.time()
ts_tour, ts_cost = tabu_search.solve(matrix, initial_tour=nn_tour, **TS_PARAMS)
exec_time = time.time() - start_time
print(f"TS hoàn thành trong {exec_time:.3f}s. Cost: {ts_cost:.0f}")

save_path = os.path.join(OUTPUT_DIR, f"{PROBLEM_NAME}_10_tabu_search.png")
visualizer.plot_tour(ts_tour, coords, f"Tabu Search Tour ({PROBLEM_NAME})\nCost: {ts_cost:.0f}", save_path)

## 5. Hoàn tất

Tất cả các hình ảnh đã được tạo và lưu vào thư mục `/results/tours/`.

Bạn có thể vào thư mục đó và mở các file `.png` để xem trực tiếp sự khác biệt về "vẻ đẹp" và trật tự của các lộ trình do từng thuật toán tạo ra.